
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-27 23:21:36.654127: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 23:21:36.869628: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 23:21:36.869761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 23:21:37.207278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 23:21:37.926708: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 23:21:37.928675: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.5,
            hidden_sizes = [[15, 7], [30, 15], [30, 15, 7]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[15, 7], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  2.4274016020000317
Stopped at epoch:  40
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.366037021000011
Stopped at epoch:  7
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.9182165729999952
Stopped at epoch:  17
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.317084938999983
Stopped at epoch:  7
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.38625407100005305
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.41242870300004597
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 7 ++++++++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_2_0,0.510663,-0.255007,0.029379,0.095632,0.091544,0.087082,0.129989,0.107274
model_2_2_1,0.518216,-0.154555,0.024680,0.124248,0.090131,0.080112,0.130618,0.103879
model_2_2_18,0.526174,0.014408,-0.035259,0.144726,0.088642,0.068388,0.138645,0.101450
model_2_2_17,0.526174,0.014408,-0.035259,0.144726,0.088642,0.068388,0.138645,0.101450
model_2_2_16,0.526174,0.014408,-0.035259,0.144726,0.088642,0.068388,0.138645,0.101450
model_2_2_15,0.526174,0.014408,-0.035259,0.144726,0.088642,0.068388,0.138645,0.101450
model_2_2_14,0.526174,0.014408,-0.035259,0.144726,0.088642,0.068388,0.138645,0.101450
model_2_2_13,0.526174,0.014408,-0.035259,0.144726,0.088642,0.068388,0.138645,0.101450
model_2_2_12,0.526174,0.014408,-0.035259,0.144726,0.088642,0.068388,0.138645,0.101450
model_2_2_11,0.526174,0.014408,-0.035259,0.144726,0.088642,0.068388,0.138645,0.101450


DataFrame salvo em ../results/metrics_2_2
+++++++++++ [2_3] | 1 ++++++++++++++++++
Elapsed time:  0.06143924100001641
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_3] | 2 ++++++++++++++++++
Elapsed time:  0.02898204699999951
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_3] | 3 ++++++++++++++++++
Elapsed time:  0.042594504000021516
Stopped at epoch:  0
1/1 [==============================] - 0s 22ms/step
+++++++++++ [2_3] | 4 ++++++++++++++++++
Elapsed time:  0.03523727499998586
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_3] | 5 ++++++++++++++++++
Elapsed time:  0.03899606500010577
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_3] | 6 ++++++++++++++++++
Elapsed time:  0.029206193999925745
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_3] | 7 ++++++++++++++++++
Elapsed time:  0.0292

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_3_0,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_22,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_21,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_20,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_19,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_18,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_17,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_16,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_15,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592
model_2_3_14,0.526174,0.516742,0.308784,0.505136,0.088642,0.113319,0.073524,0.094592


DataFrame salvo em ../results/metrics_2_3
+++++++++++ [2_4] | 1 ++++++++++++++++++
Elapsed time:  0.05079543099998318
Stopped at epoch:  0
1/1 [==============================] - 0s 25ms/step
+++++++++++ [2_4] | 2 ++++++++++++++++++
Elapsed time:  0.03849009300006401
Stopped at epoch:  0
1/1 [==============================] - 0s 29ms/step
+++++++++++ [2_4] | 3 ++++++++++++++++++
Elapsed time:  0.04302779299996473
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_4] | 4 ++++++++++++++++++
Elapsed time:  0.042667480999966756
Stopped at epoch:  0
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_4] | 5 ++++++++++++++++++
Elapsed time:  0.04779523199999858
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_4] | 6 ++++++++++++++++++
Elapsed time:  0.040919543999962116
Stopped at epoch:  0
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_4] | 7 ++++++++++++++++++
Elapsed time:  0.0305

...

DataFrame salvo em ../results/metrics_10_7
+++++++++++ [10_8] | 1 ++++++++++++++++++
Elapsed time:  0.6009427439998944
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [10_8] | 2 ++++++++++++++++++
Elapsed time:  0.6590718119996382
Stopped at epoch:  7
1/1 [==============================] - 0s 30ms/step
+++++++++++ [10_8] | 3 ++++++++++++++++++
Elapsed time:  0.36290736000000834
Stopped at epoch:  6
1/1 [==============================] - 0s 36ms/step
+++++++++++ [10_8] | 4 ++++++++++++++++++
Elapsed time:  0.3830340760000581
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [10_8] | 5 ++++++++++++++++++
Elapsed time:  0.4708336510002482
Stopped at epoch:  6
1/1 [==============================] - 0s 28ms/step
+++++++++++ [10_8] | 6 ++++++++++++++++++
Elapsed time:  0.3759084250000342
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [10_8] | 7 ++++++++++++++++++
Elapsed time:  0.319

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_10_8_24,0.572186,-0.395251,-2.532525,-0.514773,0.080034,0.231605,0.299006,0.263323
model_10_8_23,0.577422,-0.382710,-2.477305,-0.495782,0.079055,0.229523,0.294332,0.260022
model_10_8_22,0.583148,-0.369073,-2.416497,-0.474952,0.077984,0.227259,0.289185,0.256401
model_10_8_21,0.589392,-0.353980,-2.350353,-0.452166,0.076816,0.224754,0.283587,0.252440
model_10_8_20,0.596238,-0.337531,-2.277201,-0.427090,0.075535,0.222024,0.277395,0.248081
model_10_8_19,0.603685,-0.319819,-2.196758,-0.399702,0.074142,0.219083,0.270586,0.243320
model_10_8_18,0.611822,-0.300468,-2.108220,-0.369633,0.072619,0.215871,0.263092,0.238092
model_10_8_17,0.620623,-0.279684,-2.011365,-0.336935,0.070973,0.212421,0.254893,0.232408
model_10_8_16,0.630273,-0.257467,-1.903014,-0.300876,0.069168,0.208733,0.245722,0.226140
model_10_8_15,0.640686,-0.233483,-1.784826,-0.261670,0.067220,0.204752,0.235718,0.219325


DataFrame salvo em ../results/metrics_10_8
+++++++++++ [10_9] | 1 ++++++++++++++++++
Elapsed time:  0.6744947880001746
Stopped at epoch:  11
1/1 [==============================] - 0s 28ms/step
+++++++++++ [10_9] | 2 ++++++++++++++++++
Elapsed time:  0.4765932150003209
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [10_9] | 3 ++++++++++++++++++
Elapsed time:  0.3514331979999952
Stopped at epoch:  6
1/1 [==============================] - 0s 36ms/step
+++++++++++ [10_9] | 4 ++++++++++++++++++
Elapsed time:  0.3746579339999698
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [10_9] | 5 ++++++++++++++++++
Elapsed time:  0.3787987359996805
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [10_9] | 6 ++++++++++++++++++
Elapsed time:  0.36552051900025617
Stopped at epoch:  6
1/1 [==============================] - 0s 64ms/step
+++++++++++ [10_9] | 7 ++++++++++++++++++
Elapsed time:  0.40

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_10_9_0,0.785584,0.561303,0.013142,0.307687,0.040112,0.060788,0.135397,0.095898
model_10_9_24,0.793865,0.412298,-0.296200,0.084586,0.038563,0.081435,0.177839,0.126802
model_10_9_23,0.794508,0.414087,-0.291261,0.087839,0.038443,0.081187,0.177161,0.126351
model_10_9_22,0.795213,0.416084,-0.285762,0.091454,0.038311,0.080911,0.176407,0.125850
model_10_9_1,0.795711,0.551390,0.013837,0.302760,0.038218,0.062162,0.135302,0.096581
model_10_9_21,0.795982,0.418268,-0.279625,0.095475,0.038167,0.080608,0.175565,0.125293
model_10_9_20,0.796825,0.420724,-0.272789,0.099960,0.038009,0.080268,0.174627,0.124672
model_10_9_19,0.797747,0.423449,-0.265155,0.104966,0.037837,0.079890,0.173579,0.123979
model_10_9_18,0.798756,0.426470,-0.256551,0.110575,0.037648,0.079471,0.172399,0.123202
model_10_9_17,0.799847,0.429810,-0.246994,0.116807,0.037444,0.079009,0.171088,0.122338


DataFrame salvo em ../results/metrics_10_9
Testando combinacao11: Hidden Size=[30, 15, 7], regularizer=0.1, learning_rate=0.02
+++++++++++ [11_0] | 1 ++++++++++++++++++
Elapsed time:  2.307003759000054
Stopped at epoch:  20
1/1 [==============================] - 0s 25ms/step
+++++++++++ [11_0] | 2 ++++++++++++++++++
Elapsed time:  0.5617841270000099
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [11_0] | 3 ++++++++++++++++++
Elapsed time:  0.5728160090002348
Stopped at epoch:  7
1/1 [==============================] - 0s 21ms/step
+++++++++++ [11_0] | 4 ++++++++++++++++++
Elapsed time:  1.6431434549999722
Stopped at epoch:  14
1/1 [==============================] - 0s 26ms/step
+++++++++++ [11_0] | 5 ++++++++++++++++++
Elapsed time:  0.8536603840002499
Stopped at epoch:  10
1/1 [==============================] - 0s 21ms/step
+++++++++++ [11_0] | 6 ++++++++++++++++++
Elapsed time:  0.6196481540000605
Stopped at epoch:  6
1/1 [========================